## 模型融合
#### Boosting方法
##### AdaBoost:
$$f(x_i) = \sum _{i=0}^m a_mG_m(x_i) $$
二分类模型：损失函数为指数函数，基函数(共有m个)线性相加，通过分布前向算法求解    
    
其中，$G_m()$为基分类器，$a_m$为每个基函数的权重    
    
每个基分类器训练使用的数据不同之处在于每个数据的权重在每一轮会进行更新，更新原则是上一轮识别正确的权重这一轮权重降低，上一轮识别错误数据则权重增加，以保证下一个基分类器能重点训练上一次训练错误的数据。
     
整个训练过程中产生了n组不同权重的训练数据，虽然数据的值从头到晚没有变，但是因为数据的权重发生了改变，同一个数据在整个数据化中所占的比例其实一直在变化，从而导致了不同的基分类器的识别倾向不同，这一点与神经网络类似，不同的基分类器对不同的特征有不同的敏感度，当将这些基分类器组合起来之后，他们对特征就能做出相对好的预测了。    

特征树的构建选取：选取当前能达成最小整体损失的那个特征的那个值作为树的划分条件，跟损失函数的选取有较大关系
##### GBDT指数分类提升树：
$$f(x_i) = \sum _{i=0}^m T_m(x_i,θ_m) $$
二分类模型：相比于AdaBoost的不同点在于，限定了基函数为CART树，且基函数不再带有权重。损失函数仍然为指数函数，线性相加模型，通过分布前向算法求解。θ为CART树的参数    
    
其中$T_m()$代表整个模型中的一颗CART树。    
     
与AdaBoost的主要区别在于，没有了数据权重的概念，数据集中各个数据的权重都是相等且不变的，改变的是每个基函数训练时使用的数据基中target的数值，feat的值不变。    
    
下一个基函数使用的target的值为上一个整体损失函数$（L(y_i,f_m(x_i)）$的导数。

特征树的构建选取：比较所有分类情况下的指数函数的总体损失值，选取损失值最小的那一个
##### GBDT-MSE回归提升树：
$$f(x_i) = \sum _{i=0}^m T_m(x_i,θ_m) $$
与GBDT指数分类提升树基本一样，除了以下几点：    
1、损失函数使用MSE    
2、每个叶子节点需要有一个输出值$c_R$，当输入数据最终被分到这个叶子节点时，CART树将输出这个$c_R$作为结果，其中R是这颗树中叶子节点的数量，即每个叶子节点都需要有一个$c_R$。$c_R$的值根据损失函数确定，当选择这个$c_R$时，需要达到当前叶子节点的损失最小的效果。根据公式推导，损失函数为MSE时，$c_R$的值取当前叶子节点所有值的算术平均值叶子节点的损失最小。    
3、每个CART树使用的训练数据同GBDT指数分类提升树，只不过刚好$（L(y_i,f_m(x_i)）$的导数等于$y_i - y_i ^- $，也就是y的实际值和计算值的差，也叫残差
##### GBDT一般损失函数提升树：
与GBDT回归和分类树一样，只是当损失函数不再是指数函数和MSE时适用，计算过程更复杂一些
##### XGBoost：
$$f(x_i) = \sum _{i=0}^m T_m(x_i,θ_m) +λR+β1/2 \sum w_R^2$$ 
与GBDT的使用区别在于：    
1、增加了正则项：$λR+β1/2 \sum w_R^2$，其中λ和β是超参数，R是每个基树叶子节点的个数，$w_R$是每个叶子节点的输出值
2、相比于GBDT使用泰勒一阶展开公式拟合误差，XGBoost使用泰勒二阶展开公式拟合误差，准确度更高
##### LGB：
据说与XGBoost内容和算法基本一样，但是占用电脑资源更少，算起来更快。
#### Bagging方法
##### 随机森林：
假设有m个数据，需要训练n个基分类器。    
每轮从数据集中随机抽取m个数据组成一个新的数据集，每抽出一个数据就将数据放回，最终形成n个数据集，其中每个数据集中可能存在多个相同的数据。理论上来说数据集中每次约有30%以上的数据不会加入新数据集中，这部分没有加入当前数据集的数据称为oob(out of bag)数据，可以当做验证数据使用。    
每个基分类器训练之前，将从对应的数据集中随机抽取一部分特征进行训练，以增强模型的泛化能力。抽取的特征数量或者比例可以自定。        
基分类器也可以是CART树。
#### Stacking方法
待补充
#### Blending方法
待补充

In [5]:
import numpy as np
import pandas as pd
from  lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [18]:
train_data = pd.read_csv('zhengqi_train_change_feat.txt',sep='\t')
test_data = pd.read_csv('zhengqi_test_change_feat.txt',sep='\t')
train_target= train_data['target']
train_feat = train_data.drop('target',axis=1)
train_feat.info()
train_target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2876 entries, 0 to 2875
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   NEW_V0   2876 non-null   float64
 1   NEW_V1   2876 non-null   float64
 2   NEW_V2   2876 non-null   float64
 3   NEW_V3   2876 non-null   float64
 4   NEW_V4   2876 non-null   float64
 5   NEW_V5   2876 non-null   float64
 6   NEW_V6   2876 non-null   float64
 7   NEW_V7   2876 non-null   float64
 8   NEW_V8   2876 non-null   float64
 9   NEW_V9   2876 non-null   float64
 10  NEW_V10  2876 non-null   float64
 11  NEW_V11  2876 non-null   float64
 12  NEW_V12  2876 non-null   float64
dtypes: float64(13)
memory usage: 292.2 KB


0       0.175
1       0.676
2       0.633
3       0.206
4       0.384
        ...  
2871    0.235
2872    1.042
2873    0.005
2874    0.350
2875    0.417
Name: target, Length: 2876, dtype: float64

In [29]:
from sklearn.model_selection import GridSearchCV

def gds(model_name,model_params):
    
    gdsearch = GridSearchCV(model_name,model_params,cv=6,
                            scoring='neg_mean_squared_error',
                           verbose=1,return_train_score=True)
    gdsearch.fit(train_feat,train_target)
    model=gdsearch.best_estimator_
    best_index= gdsearch.best_index_
    best_score= gdsearch.best_score_
    best_params = gdsearch.best_params_
    grid_result = pd.DataFrame(gdsearch.cv_results_)
    cv_mean = abs(grid_result.loc[best_index,'mean_test_score'])
    cv_std = grid_result.loc[best_index,'std_test_score']

    print('best_index:',best_index)
    print('best_score:',best_score)
    print('best params:',best_params)
    print('grid_result:',grid_result)
    print('cv_mean:',cv_mean)
    print('cv_std:',cv_std)
    
    return model

In [32]:
#GBDT模型
from sklearn.ensemble import GradientBoostingRegressor

gbdt_params={'max_depth':[1,2,3,4,5],
             'min_samples_split':[3,4,5,6,7],
             'n_estimators':[150,200,250,300,350]}
gbdt_model = GradientBoostingRegressor()

gbdt_model = gds(gbdt_model,gbdt_params)

Fitting 6 folds for each of 125 candidates, totalling 750 fits
best_index: 31
best_score: -0.1505143313001109
best params: {'max_depth': 2, 'min_samples_split': 4, 'n_estimators': 200}
grid_result:      mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.445803      0.021724         0.002166        0.000894   
1         0.571312      0.001658         0.002161        0.000372   
2         0.718248      0.007648         0.001991        0.000009   
3         0.864856      0.010480         0.001662        0.000471   
4         1.008305      0.020577         0.001995        0.000576   
..             ...           ...              ...             ...   
120       2.237203      0.044647         0.004146        0.001074   
121       2.955857      0.062506         0.003664        0.000468   
122       3.633626      0.096389         0.003823        0.000896   
123       4.388535      0.035715         0.006815        0.004542   
124       5.104832      0.121315         0.

In [33]:
#RF模型
from sklearn.ensemble import RandomForestRegressor

rf_params={'max_depth':[1,2,3,4,5],
           'min_samples_split':[3,4,5,6,7],
           'n_estimators':[150,200,250,300,350],
           'max_features':[0.6,0.7,0.8,0.9,0.99]}

rf_model = RandomForestRegressor()

rf_model = gds(rf_model,rf_params)

Fitting 6 folds for each of 625 candidates, totalling 3750 fits
best_index: 585
best_score: -0.17766892636766707
best params: {'max_depth': 5, 'max_features': 0.9, 'min_samples_split': 5, 'n_estimators': 150}
grid_result:      mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.269815      0.010946         0.010808        0.001065   
1         0.343411      0.000933         0.013967        0.000576   
2         0.438987      0.020215         0.017459        0.000954   
3         0.535227      0.034148         0.022939        0.005400   
4         0.669044      0.025368         0.027770        0.004046   
..             ...           ...              ...             ...   
620       1.109535      0.024345         0.012634        0.000470   
621       1.487519      0.028450         0.016457        0.000509   
622       1.897430      0.041188         0.020278        0.000742   
623       2.314658      0.142965         0.025596        0.002420   
624       2.537722 

In [38]:
#XGBoost模型

from xgboost import XGBRegressor

xgb_params={'max_depth':[1,2,3,4,5],
           'reg_alpha':[1,0.1,0.01,0.001,0.0001],
           'reg_lambda':[1,0.1,0.01,0.001,0.0001],
           'n_estimators':[150,200,250,300,350],
           'learning_rate':[1,0.1,0.01,0.001,0.0001]}

xgb_model = XGBRegressor()

xgb_model = gds(xgb_model,xgb_params)

Fitting 6 folds for each of 3125 candidates, totalling 18750 fits
best_index: 826
best_score: -0.14753302374269328
best params: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 300, 'reg_alpha': 1, 'reg_lambda': 0.1}
grid_result:       mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0          0.113885      0.077147         0.002992    2.892856e-07   
1          0.080452      0.001487         0.002992    5.757255e-04   
2          0.080618      0.001064         0.002992    3.203653e-07   
3          0.082446      0.003341         0.002992    2.282684e-07   
4          0.080119      0.000470         0.002992    1.638375e-07   
...             ...           ...              ...             ...   
3120       0.656246      0.005182         0.003325    4.699374e-04   
3121       0.656246      0.002230         0.003491    4.986525e-04   
3122       0.655415      0.003706         0.003491    4.985333e-04   
3123       0.657077      0.002960         0.003491    4.985730e-